In [2]:
import datetime

print(datetime.datetime.now())

2024-09-17 16:25:07.677290


**Background and question**: I have never benchmarked an AI model. I installed a Mistral model back in 02/24 for a separate high-level idea. Can I benchmark this model under an hour?

In [1]:
import json

with open("./ARC-AGI-master/data/training/0a938d79.json") as f:
    some_file = json.load(f)
    print(some_file)

{'train': [{'input': [[0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'output': [[0, 0, 0, 0, 0, 2, 0, 8, 0, 2, 0, 8, 0, 2, 0, 8, 0, 2, 0, 8, 0, 2, 0, 8, 0], [0, 0, 0, 0, 0, 2, 0, 8, 0, 2, 0, 8, 0, 2, 0, 8, 0, 2, 0, 8, 0, 2, 0, 8, 0], [0, 0, 0, 0, 0, 2, 0, 8, 0, 2, 0, 8, 0, 2,

In [2]:
# reusing code from earlier
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

In [3]:
model_name = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        #load_in_4bit=True,
        #quantization_config=bnb_config,
        torch_dtype=torch.float16, #bfloat if quantization was working
        device_map="mps",
        #trust_remote_code=True,
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
params_7b = model.num_parameters() / 1_000_000
print(round(params_7b), "million")

7242 million


In [28]:
import os 

matches = []
train_file_names = []
for root, dirnames, filenames in os.walk("./ARC-AGI-master/data/training"):
    for filename in filenames:
        if filename.endswith('.json'):
            matches.append(os.path.join(root, filename))
            train_file_names.append(filename)
            
print(matches)

['./ARC-AGI-master/data/training/a85d4709.json', './ARC-AGI-master/data/training/c8cbb738.json', './ARC-AGI-master/data/training/8e1813be.json', './ARC-AGI-master/data/training/a699fb00.json', './ARC-AGI-master/data/training/5c2c9af4.json', './ARC-AGI-master/data/training/44f52bb0.json', './ARC-AGI-master/data/training/23581191.json', './ARC-AGI-master/data/training/94f9d214.json', './ARC-AGI-master/data/training/f9012d9b.json', './ARC-AGI-master/data/training/4258a5f9.json', './ARC-AGI-master/data/training/bdad9b1f.json', './ARC-AGI-master/data/training/d06dbe63.json', './ARC-AGI-master/data/training/8403a5d5.json', './ARC-AGI-master/data/training/67e8384a.json', './ARC-AGI-master/data/training/8731374e.json', './ARC-AGI-master/data/training/25ff71a9.json', './ARC-AGI-master/data/training/ecdecbb3.json', './ARC-AGI-master/data/training/6e19193c.json', './ARC-AGI-master/data/training/39e1d7f9.json', './ARC-AGI-master/data/training/ba97ae07.json', './ARC-AGI-master/data/training/99b1bc4

In [61]:
import pandas as pd

bunch_of_series = []
for name in train_file_names:
    with open(f"./ARC-AGI-master/data/training/{name}") as temp_file:
        some_file = json.load(temp_file)
        
        try:
            prob_ex_3_placement = some_file["train"][2]
        except IndexError:
            prob_ex_3_placement = {"input": [], "output": []}
            
        try:
            prob_ex_4_placement = some_file["train"][3]
        except IndexError:
            prob_ex_4_placement = {"input": [], "output": []}
        
        temp_series = pd.Series({
            "problem_id": name,
            "problem_example_1_input":  [some_file["train"][0]["input"]],
            "problem_example_1_output": [some_file["train"][0]["output"]],
            "problem_example_2_input":  [some_file["train"][1]["input"]],
            "problem_example_2_output": [some_file["train"][1]["output"]],
            "problem_example_3_input":  [prob_ex_3_placement["input"]],
            "problem_example_3_output": [prob_ex_3_placement["output"]],
            "problem_example_4_input":  [prob_ex_4_placement["input"]],
            "problem_example_4_output": [prob_ex_4_placement["output"]],
            "problem_test_input":       [some_file["test"][0]["input"]],
            "problem_test_output":      [some_file["test"][0]["output"]],
        })
        
        bunch_of_series.append(temp_series)

In [57]:
some_file["train"]

[{'input': [[0, 0, 5], [0, 5, 0], [5, 0, 0]],
  'output': [[3, 3, 3], [4, 4, 4], [2, 2, 2]]},
 {'input': [[0, 0, 5], [0, 0, 5], [0, 0, 5]],
  'output': [[3, 3, 3], [3, 3, 3], [3, 3, 3]]},
 {'input': [[5, 0, 0], [0, 5, 0], [5, 0, 0]],
  'output': [[2, 2, 2], [4, 4, 4], [2, 2, 2]]},
 {'input': [[0, 5, 0], [0, 0, 5], [0, 5, 0]],
  'output': [[4, 4, 4], [3, 3, 3], [4, 4, 4]]}]

In [58]:
test_df

problem_id                                        a85d4709.json
problem_example_1_input     [[[0, 0, 5], [0, 5, 0], [5, 0, 0]]]
problem_example_1_output    [[[3, 3, 3], [4, 4, 4], [2, 2, 2]]]
problem_example_2_input     [[[0, 0, 5], [0, 0, 5], [0, 0, 5]]]
problem_example_2_output    [[[3, 3, 3], [3, 3, 3], [3, 3, 3]]]
problem_example_3_input     [[[5, 0, 0], [0, 5, 0], [5, 0, 0]]]
problem_example_3_output    [[[2, 2, 2], [4, 4, 4], [2, 2, 2]]]
problem_example_4_input     [[[0, 5, 0], [0, 0, 5], [0, 5, 0]]]
problem_example_4_output    [[[4, 4, 4], [3, 3, 3], [4, 4, 4]]]
problem_test_input          [[[0, 0, 5], [5, 0, 0], [0, 5, 0]]]
problem_test_output         [[[3, 3, 3], [2, 2, 2], [4, 4, 4]]]
dtype: object

In [65]:
arc_train = pd.concat(bunch_of_series, axis=1).T
arc_train

,problem_id,problem_example_1_input,problem_example_1_output,problem_example_2_input,problem_example_2_output,problem_example_3_input,problem_example_3_output,problem_example_4_input,problem_example_4_output,problem_test_input,problem_test_output
0,a85d4709.json,"[[[0, 0, 5], [0, 5, 0], [5, 0, 0]]]","[[[3, 3, 3], [4, 4, 4], [2, 2, 2]]]","[[[0, 0, 5], [0, 0, 5], [0, 0, 5]]]","[[[3, 3, 3], [3, 3, 3], [3, 3, 3]]]","[[[5, 0, 0], [0, 5, 0], [5, 0, 0]]]","[[[2, 2, 2], [4, 4, 4], [2, 2, 2]]]","[[[0, 5, 0], [0, 0, 5], [0, 5, 0]]]","[[[4, 4, 4], [3, 3, 3], [4, 4, 4]]]","[[[0, 0, 5], [5, 0, 0], [0, 5, 0]]]","[[[3, 3, 3], [2, 2, 2], [4, 4, 4]]]"
1,c8cbb738.json,"[[[3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3], [3, 3, 3,...","[[[2, 4, 1, 4, 2], [8, 3, 3, 3, 8], [1, 3, 3, ...","[[[1, 1, 1, 1, 1, 1, 1, 1], [1, 8, 1, 8, 1, 1,...","[[[8, 3, 8], [3, 1, 3], [8, 3, 8]]]","[[[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], ...","[[[1, 4, 7, 4, 1], [4, 4, 4, 4, 4], [7, 4, 4, ...",[[]],[[]],"[[[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8...","[[[3, 8, 6, 1, 6, 8, 3], [8, 8, 8, 8, 8, 8, 8]..."
2,8e1813be.json,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1], [3, ...","[[[0, 0, 0, 0, 0, 2, 0, 0, 4, 0], [0, 5, 5, 5,...","[[[1, 2, 4], [1, 2, 4], [1, 2, 4]]]","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 2,...","[[[2, 2, 2, 2], [8, 8, 8, 8], [4, 4, 4, 4], [1...",[[]],[[]],"[[[2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0...","[[[2, 2, 2, 2, 2, 2, 2], [3, 3, 3, 3, 3, 3, 3]..."
3,a699fb00.json,"[[[1, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...","[[[1, 2, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1,...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 2, 1,...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0,...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0,...",[[]],[[]],"[[[0, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0,...","[[[0, 1, 2, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0,..."
4,5c2c9af4.json,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0, 0, 8, 0, 0, 8, 0, 0, 8, 0, 0, 0, 0, 0, 0...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3...",[[]],[[]],"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4..."
...,...,...,...,...,...,...,...,...,...,...,...
395,7b7f7511.json,"[[[1, 1, 3, 2, 1, 1, 3, 2], [1, 1, 3, 3, 1, 1,...","[[[1, 1, 3, 2], [1, 1, 3, 3], [3, 3, 1, 1], [2...","[[[4, 4, 4, 4, 4, 4], [6, 4, 8, 6, 4, 8], [6, ...","[[[4, 4, 4], [6, 4, 8], [6, 6, 8]]]","[[[2, 3], [3, 2], [4, 4], [2, 3], [3, 2], [4, ...","[[[2, 3], [3, 2], [4, 4]]]",[[]],[[]],"[[[5, 4, 5], [4, 5, 4], [6, 6, 4], [2, 6, 2], ...","[[[5, 4, 5], [4, 5, 4], [6, 6, 4], [2, 6, 2]]]"
396,8be77c9e.json,"[[[1, 1, 0], [1, 1, 1], [0, 0, 0]]]","[[[1, 1, 0], [1, 1, 1], [0, 0, 0], [0, 0, 0], ...","[[[0, 0, 0], [1, 0, 1], [1, 1, 0]]]","[[[0, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 0], ...","[[[0, 0, 0], [0, 0, 1], [0, 0, 1]]]","[[[0, 0, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], ...",[[]],[[]],"[[[0, 0, 0], [0, 0, 1], [1, 0, 0]]]","[[[0, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], ..."
397,dae9d2b5.json,"[[[4, 4, 0, 3, 3, 0], [4, 0, 0, 3, 0, 0], [0, ...","[[[6, 6, 0], [6, 0, 0], [0, 0, 6]]]","[[[4, 0, 4, 3, 3, 0], [4, 0, 0, 3, 0, 0], [0, ...","[[[6, 6, 6], [6, 0, 0], [6, 0, 6]]]","[[[0, 0, 4, 0, 3, 0], [0, 4, 4, 3, 0, 3], [4, ...","[[[0, 6, 6], [6, 6, 6], [6, 6, 6]]]","[[[4, 4, 0, 3, 0, 0], [0, 0, 0, 0, 0, 3], [4, ...","[[[6, 6, 0], [0, 0, 6], [6, 0, 0]]]","[[[0, 4, 4, 3, 0, 0], [4, 0, 0, 3, 3, 0], [0, ...","[[[6, 6, 6], [6, 6, 0], [6, 6, 0]]]"
398,2bee17df.json,"[[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [8, 0,...","[[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [8, 0,...","[[[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [2, 0,...","[[[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [2, 0,...","[[[8, 8, 8, 8, 8, 8, 8, 8, 8, 2], [8, 8, 8, 0,...","[[[8, 8, 8, 8, 8, 8, 8, 8, 8, 

In [71]:
from datasets import Dataset

arc_train_hf = Dataset.from_pandas(arc_train)

Things I have done so far:
1. Downloaded the ARC prize benchmark
2. Played around with it; I understand the data structure better now
3. Load the model and basic code
4. Convert the json training files to a dataframe
5. **The plan**
  - Tokenize the dataset
  - Finetune the model weights based on the tokenized dataset
  - Test the finetuned model on the evaluation set

(So, I couldn't do this in an hour, but am confident that I can do this in two more 
hours) 

In [73]:
# CHECKPOINT 
import datetime
print(datetime.datetime.now())

2024-09-17 17:44:29.061345
